In [ ]:
import os
from typing import Any, Callable, Dict

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langsmith import traceable
from pydantic import BaseModel, Field

In [2]:
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "sns-ai-automation-agency-poc"

GOOGLE_API_KEY = os.environ["GOOGLE_CUSTOM_SEARCH_API_KEY"]
GOOGLE_CX = os.environ["GOOGLE_CUSTOM_SEARCH_CX"]
GOOGLE_ENDPOINT = "https://www.googleapis.com/customsearch/v1"


In [3]:
@traceable(tags=["thumbnail_workflow_notebook"], metadata={"task": "thumbnail_generation"})
def langsmith_traced_function(func: Callable) -> Any:
    return func()

In [4]:
from sns_ai_automation_agency.agent.access import survey_access_information
from sns_ai_automation_agency.agent.restaurant import survey_restaurant_information
from sns_ai_automation_agency.agent.scene import run_scene_agent

In [5]:
thread_id = "test"
station_name = "横浜"
num_highlight_stations = 3
num_iterations = 2
total_seconds = 15

access_data = survey_access_information(
    station_name=station_name,
    num_highlight_stations=num_highlight_stations,
    thread_id=thread_id,
)

restaurant_data = survey_restaurant_information(
    station_name=station_name,
    num_iterations=num_iterations,
    thread_id=thread_id,
)
scene_data = run_scene_agent(
    restaurant_info=restaurant_data,
    access_info=access_data,
    total_seconds=total_seconds,
    thread_id=thread_id,
    station_name=station_name,
)

In [6]:
scene_data

{'scenes': [{'title': 'Scene 1 導入：海と夜景が日常の街',
   'content': '海と観覧車の夜景、レトロな街並み、ディープな飲み屋街までそろうエリアを、住む目線でまとめて紹介します。',
   'telop': '海と夜景が日常の街',
   'image_search_query': '横浜 みなとみらい 夜景 街並み',
   'start_sec': 0.0,
   'end_sec': 1.5},
  {'title': 'Scene 2 駅名紹介：横浜駅',
   'content': '今回紹介するのは、JR東海道本線や湘南新宿ラインなどが使える『横浜駅』エリアです。',
   'telop': '今日の住みたい駅は横浜駅',
   'image_search_query': '横浜駅 東口 西口 駅前 俯瞰',
   'start_sec': 1.5,
   'end_sec': 3.0},
  {'title': 'Scene 3 全体像：海も繁華街も近い街',
   'content': '横浜駅を起点に、海辺の高層ビル街からレトロな飲み屋街までが徒歩や1〜2駅圏内に集まり、エリアごとに雰囲気が大きく変わります。通勤のしやすさと遊び場の多さを両立したポジションです。',
   'telop': '海も繁華街も揃う横浜生活',
   'image_search_query': '横浜駅 周辺地図 みなとみらい 野毛 中華街 俯瞰',
   'start_sec': 3.0,
   'end_sec': 4.5},
  {'title': 'Scene 4 アクセス：都心への通勤・通学',
   'content': '横浜駅からは東京駅へ約25分、新宿駅へ約31分、品川駅へ約12分と、主要ターミナルに乗換なしでアクセス可能。隣の桜木町駅へも約3分で、都心通いもしやすい立地です。',
   'telop': '東京25分・新宿31分圏内',
   'image_search_query': 'JR東海道線 湘南新宿ライン 車内 路線図 横浜から東京 新宿',
   'start_sec': 4.5,
   'end_sec': 6.5},
  {'title': 'Scene 5 海辺カフェ＆夜景エリア',
 

In [7]:
i_scene_data = scene_data["scenes"][0]
title = i_scene_data["title"]
content = i_scene_data["content"]
telop = i_scene_data["telop"]

In [8]:
print(f"タイトル: {title}")
print(f"コンテント: {content}")
print(f"テロップ: {telop}")

タイトル: Scene 1 導入：海と夜景が日常の街
コンテント: 海と観覧車の夜景、レトロな街並み、ディープな飲み屋街までそろうエリアを、住む目線でまとめて紹介します。
テロップ: 海と夜景が日常の街


In [17]:
system_prompt = """
あなたは「住みたい駅の魅力」を伝えるSNSサムネイル用の画像検索クエリを生成するアシスタントです。

# 目的
- 入力された「タイトル」「コンテント」「テロップ」から、
  ユーザーが1秒で理解できるシンプルな“1シーン”を想起できる画像検索クエリを日本語で生成します。
- このクエリは、画像検索エンジン（Web画像検索）にそのまま投げる文字列を想定します。

# 入力
- station_name: 住みたい駅の名前
- title: シーンのタイトル
- content: シーンの説明文
- telop: 画面に載せるテロップ
- （コンテンツは「住みたい駅の魅力」を伝えるSNS投稿である）

# 出力フォーマット（必ずこのJSONだけを返す）
{{
  "query": string,         // 画像検索用クエリ（日本語キーワードの並び）
  "reason_core": string,   // 核となる視覚イメージの説明（短く）
  "reason_living": string, // 「住みたい街・住む目線」をどう反映したか
  "reason_simplicity": string // 1秒で伝わるように何を削ったか・絞ったか
}}

# クエリ生成ルール（ステップで考える）
1. 視覚イメージの抽出
   - content と telop から、「画像1枚で表現したいメインのモチーフ」を1〜2個に絞る。
   - 例: 「海と観覧車の夜景」「駅前と住宅街」「商店街のアーケード」など。

2. 「住みたい街」らしさの要素を追加
   - 観光感だけでなく、「住む目線」を連想させる単語を1つ以上含める。
   - 例: 「住宅街」「街並み」「駅前」「商店街」「生活」「日常」「公園」などから適切なものを選ぶ。

3. 1秒ルールに合うか確認
   - クエリから想像されるシーンが「1つの場面」にまとまっているか確認する。
   - シーンが分散しそうな場合は、もっとも伝えたいモチーフだけ残し、それ以外は削る。

4. クエリの構成
   - 4〜8語程度の日本語キーワードをスペース区切りで並べる。
   - 重要度の高い順に並べる。
   - 駅名やエリア名は、必要な場合のみ含める（例: 「横浜 みなとみらい」など）。
   - 店名・ビル名など、あまりに具体的すぎる固有名詞は基本的に避ける。

5. 理由の要約
   - reason_core: 「なぜそのモチーフ（例: 海と観覧車の夜景）を選んだか」を短く説明。
   - reason_living: 「どの単語で『住みたい街・住む』イメージを補強したか」を短く説明。
   - reason_simplicity: 「何を削って1秒で伝わるクエリにしたか」を短く説明。

# 禁止事項
- JSON以外のテキストを出力しない。
- 箇条書きやコメントなど、JSON以外のフォーマットは含めない。
"""

user_prompt = f"""
  "station_name": {station_name}駅,
  "title": {title},
  "content": {content},
  "telop": {telop}
"""

In [18]:
class SearchQueryResponse(BaseModel):
    query: str = Field(..., description="画像検索用クエリ（日本語キーワードの並び）")
    reason_core: str = Field(..., description="核となる視覚イメージの説明（短く）")
    reason_living: str = Field(..., description="「住みたい街・住む目線」をどう反映したか")
    reason_simplicity: str = Field(..., description="1秒で伝わるように何を削ったか・絞ったか")

In [19]:
def engine_function() -> Dict[str, Any]:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    llm = llm.with_structured_output(SearchQueryResponse)
    thumbnail_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("user", user_prompt),
        ]
    )

    # 構造化LLMチェーンを実行
    chain = thumbnail_prompt | llm
    thumbnail_result = chain.invoke({})
    return thumbnail_result


thumbnail_result = langsmith_traced_function(engine_function)

In [20]:
thumbnail_result

SearchQueryResponse(query='横浜 海と夜景 住宅街', reason_core='海と観覧車の夜景をメインに選び、横浜の魅力を表現。', reason_living='「住宅街」を加えることで、住む目線を強調。', reason_simplicity='観光要素を削り、日常的なシーンに絞った。')